<a href="https://colab.research.google.com/github/Gramthegrob/Capstone_Smartwatch/blob/main/LSTM_DATASET_BARU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/Improved_All_Combined_hr_rsp_binary.csv')
print ("Jumlah NaN")
print (df.isna().sum())

Jumlah NaN
Participant     0
HR             44
respr           0
Time(sec)       0
Label           0
dtype: int64


In [ ]:
df_clean = df.dropna()
print ("jumlah NaN")
print (df_clean.isna().sum())

jumlah NaN
Participant    0
HR             0
respr          0
Time(sec)      0
Label          0
dtype: int64


In [ ]:
features = df_clean[["HR", "respr"]].values  # hanya 2 fitur tersedia
labels = df_clean["Label"].values

In [ ]:
# Normalisasi fitur
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [ ]:
# Buat window time-series (128 timestep)
def create_sequences(features, labels, window_size=128):
    X, y = [], []
    for i in range(len(features) - window_size):
        X.append(features[i:i+window_size])
        y.append(labels[i+window_size])
    return np.array(X), np.array(y)

X, y = create_sequences(features_scaled, labels, window_size=128)

In [ ]:
# One-hot encode label karena output pakai softmax
y = to_categorical(y, num_classes=2)

# Split data: 70% train, 10% val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.30, stratify=y.argmax(axis=1), random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=1/3, stratify=y_temp.argmax(axis=1), random_state=42)

In [ ]:
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(128, 2)),
    Dropout(0.1),
    LSTM(100),
    Dense(100, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 128, 100)       │        41,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,902 (515.24 KB)

 Trainable params: 131,902 (515.24 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Callback untuk early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Latih model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

# Prediksi
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Evaluasi
print(classification_report(y_true_classes, y_pred_classes))


Epoch 1/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 414s 499ms/step - accuracy: 0.6699 - loss: 0.6304 - val_accuracy: 0.6722 - val_loss: 0.6194
Epoch 2/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 410s 500ms/step - accuracy: 0.6746 - loss: 0.6195 - val_accuracy: 0.6762 - val_loss: 0.6198
Epoch 3/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 443s 502ms/step - accuracy: 0.6729 - loss: 0.6207 - val_accuracy: 0.6728 - val_loss: 0.6199
Epoch 4/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 416s 471ms/step - accuracy: 0.6737 - loss: 0.6207 - val_accuracy: 0.6768 - val_loss: 0.6148
Epoch 5/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 467s 502ms/step - accuracy: 0.6754 - loss: 0.6188 - val_accuracy: 0.6839 - val_loss: 0.6121
Epoch 6/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 427s 484ms/step - accuracy: 0.6801 - loss: 0.6150 - val_accuracy: 0.6807 - val_loss: 0.6075
Epoch 7/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 416s 508ms/step - accuracy: 0.6821 - loss: 0.6116 - val_accuracy: 0.6737 - val_loss: 0.6294
Epoch 8/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 440s 506ms/step - accuracy: 0.6780 -

In [ ]:
# Callback untuk early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Latih model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,                    # dinaikkan agar lebih fleksibel
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

# Prediksi
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Evaluasi
print(classification_report(y_true_classes, y_pred_classes))

def plot_confusion_matrix(y_true, y_pred, labels=['Not Stressed', 'Stressed']):
    cm = confusion_matrix(y_true, y_pred)

    # Ekstrak nilai TN, FP, FN, TP
    tn, fp, fn, tp = cm.ravel()

    # Label untuk setiap sel
    labels_text = np.array([
        [f"TN = {tn}", f"FP = {fp}"],
        [f"FN = {fn}", f"TP = {tp}"]
    ])

    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=labels_text, fmt='', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title("Confusion Matrix with Labels")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

Epoch 1/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 902s 1s/step - accuracy: 0.6705 - loss: 0.6266 - val_accuracy: 0.6751 - val_loss: 0.6186
Epoch 2/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 897s 1s/step - accuracy: 0.6762 - loss: 0.6208 - val_accuracy: 0.6750 - val_loss: 0.6162
Epoch 3/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 893s 1s/step - accuracy: 0.6763 - loss: 0.6171 - val_accuracy: 0.6428 - val_loss: 0.6344
Epoch 4/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 924s 1s/step - accuracy: 0.6795 - loss: 0.6098 - val_accuracy: 0.6715 - val_loss: 0.6219
Epoch 5/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 905s 1s/step - accuracy: 0.6840 - loss: 0.6030 - val_accuracy: 0.6950 - val_loss: 0.5948
Epoch 6/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 898s 1s/step - accuracy: 0.6890 - loss: 0.5871 - val_accuracy: 0.6984 - val_loss: 0.5706
Epoch 7/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 924s 1s/step - accuracy: 0.6955 - loss: 0.5767 - val_accuracy: 0.7083 - val_loss: 0.5648
Epoch 8/30
820/820 ━━━━━━━━━━━━━━━━━━━━ 914s 1s/step - accuracy: 0.7009 - loss: 0.5672 - val_accu